## Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import nltk
import re 
warnings.filterwarnings('ignore')

from nltk.corpus import stopwords
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import TreebankWordTokenizer
from konlpy.tag import Okt 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from wordcloud import WordCloud
from transformers import AutoTokenizer

In [2]:
train_path = '../data/train.csv'
val_path = '../data/dev.csv'
test_path = '../data/test.csv'

train_df = pd.read_csv(train_path)
val_df = pd.read_csv(val_path)
test_df = pd.read_csv(test_path)

print(f"Train shape: {train_df.shape}")
print(f"Val shape: {val_df.shape}")
print(f"Test shape: {test_df.shape}")
train_df

Train shape: (12457, 4)
Val shape: (499, 4)
Test shape: (499, 2)


,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다...","Mr. Smith는 Dr. Hawkins에게 건강검진을 받으러 와서, 매년 검진 필...",건강검진
1,train_1,"#Person1#: 안녕하세요, Mrs. Parker. 잘 지내셨나요?\n#Pers...","Mrs. Parker가 Ricky와 함께 백신 접종을 위해 방문하였고, Dr. Pe...",백신 접종
2,train_2,"#Person1#: 저기요, 열쇠 세트 본 적 있어요?\n#Person2#: 어떤 ...",#Person1#은 열쇠 세트를 잃어버리고 #Person2#에게 찾는 것을 도와달라...,열쇠 분실
3,train_3,#Person1#: 너 여자친구 있는 거 왜 말 안 했어?\n#Person2#: 미...,#Person1#은 #Person2#가 여자친구가 있고 결혼할 예정이라는 사실을 말...,여자친구와의 결혼
4,train_4,"#Person1#: 안녕, 오늘 너무 멋져 보이네요. 저랑 춤 한 곡 추실래요?\n...","Malik은 Wen과 Nikki에게 춤을 제안하고, Wen은 발을 밟는 것을 감수하...",춤 제안
...,...,...,...,...
12452,train_12455,#Person1#: 안녕하세요. 혹시 맨체스터에서 오신 Mr. Green 맞으신가요...,Tan Ling은 흰머리와 수염이 특징인 Mr. Green을 맞이하여 호텔로 안내합...,호텔 안내
12453,train_12456,"#Person1#: Mister Ewing이 우리 회의장에 4시에 오라고 했지, 맞...",#Person1#과 #Person2#는 Mister Ewing의 요청에 따라 회의장...,회의 준비
12454,train_12457,#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶...,#Person2#는 #Person1#의 도움으로 5일 동안 소형차를 대여합니다.,차량 대여
12455,train_12458,#Person1#: 너 오늘 좀 기분 안 좋아 보인다? 무슨 일 있어?\n#Pers...,#Person2#의 어머니가 직장을 잃으셨다. #Person2#는 어머니가 우울해하...,실직과 대처


## Preprocessing

In [3]:
train = train_df.copy()
val = val_df.copy()
test = test_df.copy()

test

,fname,dialogue
0,test_0,"#Person1#: Ms. Dawson, 받아쓰기 좀 부탁드려야겠어요. \n#Per..."
1,test_1,#Person1#: 드디어 왔네! 뭐가 이렇게 오래 걸렸어?\n#Person2#: ...
2,test_2,"#Person1#: Kate, 여기서 일어난 일을 믿기 힘들 거야.\n#Person..."
3,test_3,"#Person1#: 생일 축하해, 이거 너를 위한 선물이야, Brian.\n#Per..."
4,test_4,#Person1#: 이 올림픽 공원 정말 크다! \n#Person2#: 맞아. 지금...
...,...,...
494,test_495,"#Person1#: 얘, Charlie, 학교 끝나고 우리 집에 와서 나랑 비디오 ..."
495,test_496,#Person1#: 어떻게 시골 음악에 관심을 갖게 되었어요?\n#Person2#:...
496,test_497,"#Person1#: 저기, Alice. 여기는 처음 와봤어요. 어떻게 기계를 사용하..."
497,test_498,#Person1#: Matthew? 안녕! \n#Person2#: Steve! 진짜...


- Normalize (정규화)

In [4]:
def normalize_text(text):
    text = text.str.strip()  # 앞뒤 공백 제거
    text = text.str.lower()  # 소문자 변환
    text = text.str.replace(r"[^a-zA-Z가-힣\s]", "", regex=True)  # 한글, 영어, 공백 제외 모두 제거
    text = text.str.replace(r"\s+", " ", regex=True).str.strip()  # 연속 공백 제거
    return text

train['dialogue'] = normalize_text(train['dialogue'])
print(f"{'-' * 15} Normalize before{'-' * 15}{train_df[['dialogue']].head()}\n")
print(f"{'-' * 15} Normalize after{'-' * 15}{train[['dialogue']].head()}")

--------------- Normalize before---------------                                            dialogue
0  #Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다...
1  #Person1#: 안녕하세요, Mrs. Parker. 잘 지내셨나요?\n#Pers...
2  #Person1#: 저기요, 열쇠 세트 본 적 있어요?\n#Person2#: 어떤 ...
3  #Person1#: 너 여자친구 있는 거 왜 말 안 했어?\n#Person2#: 미...
4  #Person1#: 안녕, 오늘 너무 멋져 보이네요. 저랑 춤 한 곡 추실래요?\n...

--------------- Normalize after---------------                                            dialogue
0  person 안녕하세요 mr smith 저는 dr hawkins입니다 오늘 무슨 일...
1  person 안녕하세요 mrs parker 잘 지내셨나요 person 안녕하세요 d...
2  person 저기요 열쇠 세트 본 적 있어요 person 어떤 종류의 열쇠요 per...
3  person 너 여자친구 있는 거 왜 말 안 했어 person 미안해 네가 알고 있...
4  person 안녕 오늘 너무 멋져 보이네요 저랑 춤 한 곡 추실래요 person 고...


- Stopwords (불용어)

In [5]:
# stopwords list
stopwords = ['person1', 'person2', 'mr', 'mrs', 'dr', 'ms', '네', '예', '응', '어', '음', '아', '그래', '저기', '혹시', '근데', '그런데']
okt = Okt()

# 형태소 분석 + 불용어 제거 함수
def preprocess_text(text, stopwords_list):
    tokens = okt.pos(text, norm=True, stem=True)  # 형태소 분석
    # 명사, 동사, 형용사만 추출 & 불용어 제거
    filtered_tokens = [
        word for word, pos in tokens
        if pos in ['Noun', 'Verb', 'Adjective'] and word not in stopwords_list
    ]
    return ' '.join(filtered_tokens)  # 또는 return filtered_tokens

train['processed_dialogue'] = train['dialogue'].apply(lambda x: preprocess_text(x, stopwords))
train[['dialogue', 'processed_dialogue']]

,dialogue,processed_dialogue
0,person 안녕하세요 mr smith 저는 dr hawkins입니다 오늘 무슨 일...,안녕하다 저 이다 오늘 무슨 일로 오다 건강검진 받다 오다 년 동안 검진 안 받다 ...
1,person 안녕하세요 mrs parker 잘 지내셨나요 person 안녕하세요 d...,안녕하다 자다 지내다 안녕하다 자다 지내다 있다 저희 둘 백신 맞다 오다 알다 백신...
2,person 저기요 열쇠 세트 본 적 있어요 person 어떤 종류의 열쇠요 per...,열쇠 세트 보다 적 있다 어떻다 종류 열쇠 열쇠 다섯 개 작다 발 장식 달다 있다 ...
3,person 너 여자친구 있는 거 왜 말 안 했어 person 미안해 네가 알고 있...,너 여자친구 있다 거 왜 말 안 하다 미안하다 알 있다 줄 알다 사랑 하다 사람 있...
4,person 안녕 오늘 너무 멋져 보이네요 저랑 춤 한 곡 추실래요 person 고...,안녕 오늘 멋지다 보이다 저 춤 하다 곡 추다 고맙다 저 춤 자다 못 추다 괜찮다 ...
...,...,...
12452,person 안녕하세요 혹시 맨체스터에서 오신 mr green 맞으신가요 perso...,안녕하다 맨체스터 오신 맞다 가요 알아보다 주다 기쁘다 테 렉스 머리 하얗다 수염 ...
12453,person mister ewing이 우리 회의장에 시에 오라고 했지 맞지 pers...,이 우리 회의장 시 오라 하다 맞다 늦다 말 하다 우리 이스트 요크 지점 몇몇 사람...
12454,person 오늘 어떻게 도와드릴까요 person 차를 빌리고 싶어요 person ...,오늘 어떻다 도 드릴 끄다 차 빌리다 싶다 어떻다 차갑다 있다 보다 대형차 중형차 ...
12455,person 너 오늘 좀 기분 안 좋아 보인다 무슨 일 있어 person 엄마가 어...,너 오늘 좀 기분 안 좋다 보이다 무슨 일 있다 엄마 어제 직장 잃다 안타깝다 나다...


- Tokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
sample_texts = train['processed_dialogue'].head().tolist()
max_length = 512

# 토크나이저 적용 (패딩/트렁케이션 포함)
encoded = tokenizer(
    sample_texts,
    max_length=max_length,    # 원하는 시퀀스 길이
    padding='max_length',     # max_length만큼 패딩
    truncation=True,          # max_length 초과시 자름
    return_tensors='pt'       # PyTorch 텐서 반환 (필요시)
)

print(encoded['input_ids'].shape)  # (배치, 시퀀스길이)
print(tokenizer.convert_ids_to_tokens(encoded['input_ids'][0]))

torch.Size([5, 512])
['[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]